#### <center>Intermediate Python and Software Enginnering</center>


## <center>Section 08 - Concurrency - Exercise Solutions</center>


### <center>Innovation Scholars Programme</center>
### <center>King's College London, Medical Research Council and UKRI <center>

### Need for locking

**Exercise 1**: Write two functions that operate on a global variable 'value'. One function adds 1 to value 'iterations' times. One function subtracts 1 to value 'iterations' times.  We need to use the given lock value for a reason, be sure to use it correctly in your code. What should the output be?

What happens when you set iterations to be 1000, 10000, 100000, 1000000? Run each iteration count a number of times and look at the results. Why do you think the behaviour changes?


In [2]:
import threading as t

value = 0
iterations = 1000
lock = t.Lock()

def adding():
    global value
    for _ in range(iterations):
        with lock:
            value += 1

def subtracting():
    global value
    for _ in range(iterations):
        with lock:
            value -= 1

t0 = t.Thread(target=adding)
t1 = t.Thread(target=subtracting)
t0.start()
t1.start()
t0.join()
t1.join()
print(value)


# Having experimented with iteration lengths, use locks to make this safe

0


### Producer-Consumer

This is a very common pattern composed of one or more producer threads which place data into a data structure and one or more consumer threads which remove it. Both may be implementing complex behaviour but the key pattern is that they communicate through the shared data structure. They must be implemented in a way which is thread-safe and controllable, that is the threads will start and complete in a controlled way.

**Exercise 2**: implement the functions below to define producer and consumer threads

In [5]:
import threading as t

queue=[]
lock=t.Lock()
import random

SENTINEL=object() # this is one way to indicate a stopping condition


def producer(num_values):
    for n in range(num_values):
        with lock:
            val = random.randint(10,99)
            print('Producing',val)
            queue.append(val)
            
    with lock:
        queue.append(SENTINEL) # assumes only one producer, how would we handle multiples?
            
        
def consumer():
    while True:
        with lock:
            if len(queue)>0:
                val=queue.pop(0)
                if val is SENTINEL:
                    break
                    
                print('Consuming',val)
            
            
t0 = t.Thread(target=producer,args=(10,))
t1 = t.Thread(target=consumer)
t0.start()
t1.start()
t0.join()
t1.join()

Producing 30
Producing 54
Producing 51
Producing 69
Producing 57
Producing 58
Producing 81
Producing 52
Producing 58
Producing 44
Consuming 30
Consuming 54
Consuming 51
Consuming 69
Consuming 57
Consuming 58
Consuming 81
Consuming 52
Consuming 58
Consuming 44


### The Producer / Consumer Queue

We can use a producer / consumer queue whenever we have two (or more) threads in the following relationship:
* One (or more) producer threads doing work and putting the results of the work into a queue
* One (or more) threads that are consuming the work done by the producer threads by taking an item of work off the queue

In a basic producer / consumer queue, we can have a lock that is shared by producers and consumers. The lock must be obtained before:
* Pushing an item onto the queue
* Popping an item off the queue

**Exercise 3**: Write a producer consumer Queue by implementing the following methods which use an internal lock to synchronize. Use this type to re-implement your code for the above exercise. For this exercise and the following exercises, run each stage a number of times so that you can see the effects of running threaded code. 

In [6]:
import threading as t
import random

class Queue:
    def __init__(self):
        self.lock = t.Lock()
        self.queue = list()
        self.polled_empty = 0
        self.longest_queue = 0
    
    def qpush(self, value):
        with self.lock:
            self.queue.insert(0, value)
        
    def qpop(self):
        while True:
            with self.lock:
                if len(self.queue) > 0:
                    self.longest_queue = max(len(self.queue), self.longest_queue)
                    return self.queue.pop()
                else:
                    self.polled_empty += 1
                    
    def get_polled_empty(self):
        with self.lock:
            return self.polled_empty
        
    def get_longest_queue(self):
        with self.lock:
            return self.longest_queue
        

q = Queue()
iterations = 10000
r = random.Random(12345678)

def produce():
    for i in range(iterations):
        rsum = sum([r.random() for _ in range(10)])
        q.qpush(rsum)

def consume():
    sum = 0
    for i in range(iterations):
        sum += q.qpop()
    print(sum)

t0 = t.Thread(target=produce)
t1 = t.Thread(target=consume)
t0.start()
t1.start()
t0.join()
t1.join()
print('polled_empty =', q.get_polled_empty())
print('longest_queue =', q.get_longest_queue())
print("Done!")

50084.88256199735
polled_empty = 215234
longest_queue = 3442
Done!


**Exercise 4**: Add a get_polled_empty() method which returns how many times the queue was checked when empty by the consumer. Add a get_longest_queue() method that returns the longest that the queue was when checked by the consumer.

**Exercise 5**: Notice that the consumer (almost) never has time to drain the queue. Make the producer use random to calculate a more expensive value so that the consumer has time to catch up.  

### Dining philosophers
A philosopher needs two forks to eat (one for each hand). A philosopher can only pick up a single fork in an action, and so picking up two forks requires two actions. 

**Exercise 6**: Implement the `Fork` and `Philosopher` classes.

In [ ]:
import threading as t


class Fork:
    def __init__(self, index):
        self.lock = t.Lock()
        self.index = index
        
    def get_fork(self, philosopher):
        self.lock.acquire()
        print('philosopher {} got_fork {}'.format(philosopher, self.index))
        
    def release_fork(self, philosopher):
        self.lock.release()
        print('philosopher {} released_fork {}'.format(philosopher, self.index))
        
class Philosopher:
    def __init__(self, index, forks, count):
        self.index = index
        self.forks = forks
        self.count = count
        
    def eat(self):
        print('philosopher {} eats!'.format(self.index))
        
    def go(self):
        self.cur = 0
        while self.cur < self.count:
            self.forks[self.index].get_fork(self.index)
            next_fork = (self.index + 1) % len(self.forks)
            self.forks[next_fork].get_fork(self.index)
            self.eat()
            self.forks[next_fork].release_fork(self.index)
            self.forks[self.index].release_fork(self.index)
            self.cur += 1
        

count = 10
forks = [Fork(i) for i in range(0, 5)]

philosophers = [Philosopher(i, forks, count) for i in range(0, 5)]

threads = []
for p in philosophers:
    th = t.Thread(target=p.go)
    threads.append(th)
    th.start()

for th in threads:
    th.join()
    
print('done!')

philosopher 0 got_fork 0philosopher 1 got_fork 1
philosopher 2 got_fork 2
philosopher 3 got_fork 3
philosopher 4 got_fork 4



You will likely have noticed that your code doesn't always complete. This can happen when every philosopher has grabbed their left fork and no-one is letting go of the fork they grabbed. There are several ways to resolve this:
1. Release the fork after a certain amount of time
2. Always obtain forks in a particular precedence order. This means that one of the philosophers will grab their right fork before the left fork